In [11]:
import time
start = time.time()

In [12]:
from langchain.vectorstores import Pinecone
import pinecone
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENV = os.getenv('PINECONE_ENV')

# Initializing Pinecone Vector DB
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# Pinecone Vector DB index name
# index_name = 'it-case-files'
index_name = 'slp'
index = pinecone.Index(index_name)
    

In [13]:
embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [14]:
compressor = CohereRerank(top_n=5)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=docsearch.as_retriever()
)

In [15]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n" + d.page_content + "\n" +str(d.metadata) for i, d in enumerate(docs)]
        )
    )

In [16]:
# completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-4',
    temperature=0.5
)

In [17]:
query = "What will be the consequences of not deducting/depositing TDS?"

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents(query)
pretty_print_docs(compressed_docs)

In [19]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=compression_retriever )

In [20]:
qa.run(query=query)

'If the tax is not deducted or deposited under s. 194C or other provisions of the IT Act, the assessee will face the consequences provided under Chapter XVII of the Act. This could involve penalties or interest. If the TDS is not deposited, it is a clear contradiction of the provisions of the Act and the relevant provisions are attracted. Even if the assessee is entitled to claim a refund or get it adjusted against the tax liability, they cannot withhold the TDS deducted. If they do, the provisions of s. 40(a)(ia) are attracted. Furthermore, failure to deposit TDS into the Government account is not accepted as a reasonable cause for non-issue of certificate in Form No. 16A, and penalty under s. 272A(2)(g) is leviable for default in issuing TDS certificate even if tax is not deposited.'

MAP_REDUCE
'Yes, the deductee should get the credit of TDS even if the deductor did not deposit TDS with the government. The text states that the assessee should not be denied the benefit of TDS on the sole ground that the tax so deducted has not been paid to the credit of the Central Government.'

MAP_RERANK

"Yes, the deductee should still get the credit of TDS even if the deductor did not deposit the TDS with the government. It's the department's responsibility to recover such amount from the deductor."

REFINE
'While in theory, a deductee should be able to claim the credit for Tax Deducted at Source (TDS) even if the deductor has not deposited the TDS with the government, this may not always be the case in practice. The credit for TDS is typically granted based on the record of TDS payments held by the government. Consequently, if the deductor fails to deposit the TDS, it will not appear in these records and the deductee may encounter difficulties in claiming the credit. Therefore, it is crucial for the deductee to ensure the deductor is making timely TDS deposits with the government.'


STUFF
'Yes, the deductee should get the credit of TDS even if the deductor did not deposit the TDS with the government. The liability to pay income-tax if deducted at source is on the employer, or the deductor. If the deductor fails to deposit the TDS, the deductee should not be made to suffer for the illegalities committed by the deductor. It is the responsibility of the department to recover such amount from the deductor.'